In [45]:
import pandas as pd
import numpy as np

In [46]:
df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/RNAbinding/Main_data.txt", sep=">", header=None)
df = df.drop([1],1).dropna().reset_index().drop(['index'],1)

In [47]:
# all_filenames = ['/content/drive/My Drive/Colab Notebooks/RNAbinding/PSSMmatrices_train/final_result ('+str(i)+').csv' for i in range(69)]
# all_filenames[0] = all_filenames[0][:-8]+all_filenames[0][-4:]
# combined_csv_data = pd.concat([pd.read_csv(f) for f in all_filenames])
# combined_csv_data.to_csv("/content/drive/My Drive/Colab Notebooks/RNAbinding/rawTrain.csv")

In [48]:
s = ""
for index, i in enumerate(df[0]):
    if index&1 == 1:
        s += i

In [49]:
z = 0
o = 0
for i in s:
    if i == '1':
        o+=1
    else:
        z+=1
print("Zeroes: "+str(z)+" Ones: "+str(o))

Zeroes: 10827 Ones: 3747


In [50]:
X = pd.read_csv("/content/drive/My Drive/Colab Notebooks/RNAbinding/rawTrain.csv").drop('Unnamed: 0',1)
y = pd.DataFrame([i for i in s])

In [51]:
from sklearn.metrics import plot_confusion_matrix, matthews_corrcoef
from sklearn.model_selection import KFold
from sklearn.feature_selection import RFE
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

In [52]:
# y.shape

In [53]:
kf = KFold()

In [54]:
X1 = X.to_numpy()
y1 = y.to_numpy().reshape(1,len(y))[0]

In [55]:
import math as mt
X1 = (np.log2(X1/0.05))/mt.log2(10)
# X1 = (np.log2(X1/0.05))
X1 = 1/(1+2**-X1)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in log2
  


In [56]:
X1.shape

(14574, 357)

In [57]:
clf = SVC(kernel="linear", class_weight='balanced')
# sel_chi2 = SelectKBest(f_classif, k=320)
sel_chi2 = RFE(estimator=clf, n_features_to_select=320, step=25)

In [ ]:
for train_index, test_index in kf.split(X):
    X_train, X_test = X1[train_index], X1[test_index]
    y_train, y_test = y1[train_index], y1[test_index]

    X_train_chi2 = sel_chi2.fit_transform(X_train, y_train)
    selector = sel_chi2.get_support(True)
    # print(selector)
    # print(str(X_train.shape)+" "+str(X_train_chi2.shape))
    X_test_chi2 = sel_chi2.transform(X_test)

    clf.fit(X_train_chi2, y_train)
    print(clf.score(X_test_chi2,y_test))

    y_pred = clf.predict(X_test_chi2)
    print(matthews_corrcoef(y_test, y_pred))
    disp = plot_confusion_matrix(clf, X_test_chi2, y_test)
    print(disp.confusion_matrix)

0.7506003430531732
0.2607016734203143
[[1946  493]
 [ 234  242]]
0.7224699828473413
0.32036734892441054
[[1745  605]
 [ 204  361]]
0.748885077186964
0.34105883985638674
[[1876  577]
 [ 155  307]]
0.6878216123499142
0.398274978254021
[[ 980  641]
 [ 269 1025]]


TESTING

In [ ]:
# test = pd.read_csv("/content/drive/My Drive/Colab Notebooks/RNAbinding/Test_data.txt", sep=">", header=None)
# test = test.drop([1],1).dropna().reset_index().drop(['index'],1)

In [ ]:
# all_filenames = ['/content/drive/My Drive/Colab Notebooks/RNAbinding/PSSMmatrices_test/final_result ('+str(i)+').csv' for i in range(17)]
# all_filenames[0] = all_filenames[0][:-8]+all_filenames[0][-4:]
# combined_csv_data = pd.concat([pd.read_csv(f) for f in all_filenames])
# combined_csv_data.to_csv("/content/drive/My Drive/Colab Notebooks/RNAbinding/rawTest.csv")

In [ ]:
X_test = pd.read_csv("/content/drive/My Drive/Colab Notebooks/RNAbinding/rawTest.csv").drop('Unnamed: 0',1)
X_test = X_test.to_numpy()

In [ ]:
X_test = (np.log2(X_test/0.05))/mt.log2(10)
# X_test = (np.log2(X_test/0.05))
X_test = 1/(1+2**-X_test)

In [ ]:
res = clf.predict(sel_chi2.transform(X_test))

In [ ]:
import csv

In [ ]:
f = open("/content/drive/My Drive/Colab Notebooks/RNAbinding/ans.csv", "w")
writer = csv.DictWriter(f, fieldnames=["Id", "Expected"])
writer.writeheader()
f.close()

In [ ]:
id = 0
with open('/content/drive/My Drive/Colab Notebooks/RNAbinding/ans.csv', 'a', newline='') as f:
    writer = csv.writer(f)
    for row in res:
        writer.writerow([id, row])
        id = id + 1
f.close()